<a href="https://colab.research.google.com/github/meghrajpardesi/Image_classification-cloths-/blob/master/Greendeck_assignment(Machine_learning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
""""   All Imports are in this section"""
#  packages for general use
import os
import numpy as np
import pandas as pd
import urllib
from csv import reader
# --------------
#  packages for google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image


import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm


In [0]:
# mounting the google

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


#Dataset from the Zip File



> Link for dress_pattern.csv file
link: https://drive.google.com/file/d/1cHJ6QC8L4ztbYUP454Z_bbc06LxkGBOI/view?usp=sharing



> id = 1cHJ6QC8L4ztbYUP454Z_bbc06LxkGBOI


> name = dress_patterns.csv


> Link for dataset's Zip file link : https://drive.google.com/open?id=1oiYMvlxfcxSbbPqYW8ctifmBrJjvzb8Z


> ID of the file = 1oiYMvlxfcxSbbPqYW8ctifmBrJjvzb8Z


>  name = assignment.zip


In [0]:
download = drive.CreateFile({'id': '1oiYMvlxfcxSbbPqYW8ctifmBrJjvzb8Z'})
download.GetContentFile('assignment.zip')


In [4]:
# unzipping the downloaded zip file
!unzip assignment.zip

Archive:  assignment.zip
   creating: assignment/
  inflating: assignment/722119b321e04edd90a0042e874fc9e3.png  
  inflating: assignment/711e15ac11f54ae793c4a4dd244a8ce5.png  
  inflating: assignment/6bb2791d8a1244cb8a3eebfe82f8ef79.png  
  inflating: assignment/eea30d461a2d420495b8f8795363e6a7.png  
  inflating: assignment/f41cb139003a40c98d890a9364473cb6.png  
  inflating: assignment/25680041670a4dd98cd19d2777aeb097.png  
  inflating: assignment/ef9ce30fac474a74a645464920b00e83.png  
  inflating: assignment/b173f7850ea742529a8e9d6a3ad0b70e.png  
  inflating: assignment/4086730a4409413c8597075c955f784b.png  
  inflating: assignment/5334dda35ca4456991badc5c37d1659f.png  
  inflating: assignment/1a5eeacbad8847109cb0afd5185fc303.png  
  inflating: assignment/d92da54d4365400785b0cc25960f21e7.png  
  inflating: assignment/07fabfc4daaf4f16899841b83c9b7dbd.png  
  inflating: assignment/9aa082b4bd3c4f5690cf5e398aec062e.png  
  inflating: assignment/eb22480c93594b14ab80738ee5b700e2.png  
  inf

# Making dataset using url from data_patterns.csv

In [0]:
download = drive.CreateFile({'id': '1cHJ6QC8L4ztbYUP454Z_bbc06LxkGBOI'})
download.GetContentFile('dress_patterns.csv')

In [0]:
mkdir dataset

In [9]:

csv_filename = 'dress_patterns.csv' # csv file contains urls
dst_dir = '/content/dataset/' # destination directory


def getImages(csv_file,loc):
    img_counts = 0
    fail_counts = 0
    with tqdm(open(csv_file, 'r')) as csv_file:
        for line in reader(csv_file):
            # line[1] is dedicated dir name
            # line[0] is id as name 
            url = line[3]
            name = url.rsplit('/')[-1].split('.')[0] +'.png'
            filename = loc + name
            
            try :
                urllib.request.urlretrieve(url, filename)
                img_counts += 1
            except  Exception as e:
                fail_counts += 1
                print(str(e))
    print('\n Done')
    print(f"Images downloaded : {img_counts}")
    print(f"Images failed to download {fail_counts}")
                         
getImages(csv_filename,dst_dir)


2it [00:01,  1.90it/s]


# Preparing data

In [18]:
# this will also going to used in model
IMG_WIDTH = 50
IMG_HEIGHT = 50
IMG_CHENNEL = 1
IMG_DIR = '/content/assignment'
# IMG_DIR = '/content/dataset'

# list of images data for training 


# function to create a train
def makeImageData(file,img_data_dir,IMG_WIDTH = 50,IMG_HEIGHT = 50,IMG_CHENNEL = 1):
    df = pd.read_csv(file)
    train_image = []
    for i in tqdm(range(0,df.shape[0])):
        url = df['image_url'][i]
        name = url.rsplit('/')[-1].split('.')[0]
        img_path = img_data_dir +'/'+name

        img = image.load_img(img_path+'.png', target_size=(IMG_WIDTH,IMG_HEIGHT,IMG_CHENNEL),color_mode = "grayscale")
        img = image.img_to_array(img)
        img = img/255
        train_image.append(img)

    X = np.array(train_image)
    np.save('X_images.npy',X)
    return X


# X is list containing all processed images 
X = makeImageData(csv_filename,IMG_DIR,IMG_WIDTH,IMG_HEIGHT,IMG_CHENNEL)


100%|██████████| 15702/15702 [02:56<00:00, 81.85it/s]


In [0]:
# representing string as numenral for better understanding
def createLabelList(file,label_col):
    catDict = {}
    y =[]
    df = pd.read_csv(file)
    y_temp = df[label_col].unique()

    for i,cat in enumerate(y_temp):
        catDict[cat] = i

    temp = df[label_col].values

    for name in temp:
        y.append(catDict[name])

    y = to_categorical(y)
   
    np.save('y_labels.npy',y)

    return y,catDict

y,category_dict = createLabelList(csv_filename,'category')


In [0]:

def splitdataset(X,y,split_size =0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=split_size)
    return  X_train, X_test, y_train, y_test

split_size =0.2 # ratio in which data devides
# to Create train and test dataset
X_train, X_test, y_train, y_test = splitdataset(X,y,split_size)


#  To further divide X-train data in train and validation dataset.
X_train, X_val, y_train, y_val = splitdataset(X_train,y_train ,split_size)


# Creating Model

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', name='conv_1', 
                 input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHENNEL)))
model.add(MaxPooling2D((2, 2), name='maxpool_1'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='conv_2'))
model.add(MaxPooling2D((2, 2), name='maxpool_2'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='conv_3'))
model.add(MaxPooling2D((2, 2), name='maxpool_3'))
# model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='conv_4'))
# model.add(MaxPooling2D((2, 2), name='maxpool_4'))
model.add(Flatten())
model.add(Dropout(0.5))
# model.add(Dense(512, activation='relu', name='dense_1'))
model.add(Dense(128, activation='relu', name='dense_2'))
model.add(Dense(17, activation='softmax', name='output'))

model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [0]:
#  training the model
model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val))


Train on 10048 samples, validate on 2513 samples
Epoch 1/100
10048/10048 [==============================] - 3s 285us/step - loss: 1.6903 - acc: 0.5344 - val_loss: 1.6108 - val_acc: 0.5651
Epoch 2/100
10048/10048 [==============================] - 2s 226us/step - loss: 1.5768 - acc: 0.5654 - val_loss: 1.5281 - val_acc: 0.5889
Epoch 3/100
10048/10048 [==============================] - 2s 229us/step - loss: 1.5317 - acc: 0.5815 - val_loss: 1.5251 - val_acc: 0.5937
Epoch 4/100
10048/10048 [==============================] - 2s 234us/step - loss: 1.5051 - acc: 0.5941 - val_loss: 1.5261 - val_acc: 0.6021
Epoch 5/100
10048/10048 [==============================] - 2s 236us/step - loss: 1.4767 - acc: 0.5983 - val_loss: 1.4780 - val_acc: 0.6057
Epoch 6/100
10048/10048 [==============================] - 2s 234us/step - loss: 1.4471 - acc: 0.6057 - val_loss: 1.4813 - val_acc: 0.6005
Epoch 7/100
10048/10048 [==============================] - 2s 231us/step - loss: 1.4176 - acc: 0.6098 - val_loss: 1.4

In [23]:
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [24]:
model.evaluate(x=X_test, y=y_test, batch_size=None, verbose=1,)

3141/3141 [==============================] - 0s 101us/step


[2.893429580405805, 0.5749761222920119]

In [56]:
x = np.expand_dims(X_test[1006], axis=0)
result = model.predict_classes(x)

print(result)

1/1 [==============================] - 0s 3ms/step
[1]


In [54]:
print(y_test[1006])

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
